In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import re
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
cols = ['sentiment','id','date','query_string','user','text']

In [3]:
df = pd.read_csv("training.1600000.processed.noemoticon.csv",header=None, names=cols, encoding = "ISO-8859-1")

In [4]:
df.head()

,sentiment,id,date,query_string,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
sentiment       1600000 non-null int64
id              1600000 non-null int64
date            1600000 non-null object
query_string    1600000 non-null object
user            1600000 non-null object
text            1600000 non-null object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [6]:
df.sentiment.value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

In [7]:
df.drop(['id','date','query_string','user'],axis=1,inplace=True)

In [8]:
df.head()


,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [9]:
df[df.sentiment == 0].head(10)

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
5,0,@Kwesidei not the whole crew
6,0,Need a hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,@Tatiana_K nope they didn't have it
9,0,@twittera que me muera ?


In [10]:
df[df.sentiment == 4].head(10)

,sentiment,text
800000,4,I LOVE @Health4UandPets u guys r the best!!
800001,4,im meeting up with one of my besties tonight! ...
800002,4,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800003,4,Being sick can be really cheap when it hurts t...
800004,4,@LovesBrooklyn2 he has that effect on everyone
800005,4,@ProductOfFear You can tell him that I just bu...
800006,4,@r_keith_hill Thans for your response. Ihad al...
800007,4,"@KeepinUpWKris I am so jealous, hope you had a..."
800008,4,"@tommcfly ah, congrats mr fletcher for finally..."
800009,4,@e4VoIP I RESPONDED Stupid cat is helping me ...


In [11]:
df['sentiment'] = df['sentiment'].map({0: 'Negative', 4: 'Positive'})

In [12]:
df.sentiment.value_counts()

Negative    800000
Positive    800000
Name: sentiment, dtype: int64

In [13]:
df['pre_clean_len'] = [len(t) for t in df.text]

In [14]:
df[df.pre_clean_len > 140].head(10)

,sentiment,text,pre_clean_len
213,Negative,Awwh babs... you look so sad underneith that s...,142
226,Negative,Tuesdayï¿½ll start with reflection ï¿½n then a...,141
279,Negative,Whinging. My client&amp;boss don't understand ...,145
343,Negative,@TheLeagueSF Not Fun &amp; Furious? The new ma...,145
400,Negative,#3 woke up and was having an accident - &quot;...,144
464,Negative,"My bathtub drain is fired: it haz 1 job 2 do, ...",146
492,Negative,"pears &amp; Brie, bottle of Cabernet, and &quo...",150
747,Negative,Have an invite for &quot;Healthy Dining&quot; ...,141
957,Negative,Damnit I was really digging this season of Rea...,141
1064,Negative,Why do I keep looking...I know that what I rea...,141


In [15]:
df.text[279]

"Whinging. My client&amp;boss don't understand English well. Rewrote some text unreadable. It's written by v. good writer&amp;reviewed correctly. "

In [16]:
from bs4 import BeautifulSoup
example1 = BeautifulSoup(df.text[279], 'lxml')
print(example1.get_text())

Whinging. My client&boss don't understand English well. Rewrote some text unreadable. It's written by v. good writer&reviewed correctly. 


In [17]:
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

In [18]:
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [19]:
nums = [0,400000,800000,1200000,1600000]

In [20]:
%%time
print("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []
for i in range(nums[0],nums[1]):
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[1] ))                                                                   
    clean_tweet_texts.append(tweet_cleaner(df['text'][i]))

Cleaning and parsing the tweets...

Tweets 10000 of 400000 has been processed
Tweets 20000 of 400000 has been processed
Tweets 30000 of 400000 has been processed
Tweets 40000 of 400000 has been processed
Tweets 50000 of 400000 has been processed
Tweets 60000 of 400000 has been processed
Tweets 70000 of 400000 has been processed
Tweets 80000 of 400000 has been processed
Tweets 90000 of 400000 has been processed
Tweets 100000 of 400000 has been processed
Tweets 110000 of 400000 has been processed
Tweets 120000 of 400000 has been processed
Tweets 130000 of 400000 has been processed
Tweets 140000 of 400000 has been processed
Tweets 150000 of 400000 has been processed
Tweets 160000 of 400000 has been processed
Tweets 170000 of 400000 has been processed
Tweets 180000 of 400000 has been processed
Tweets 190000 of 400000 has been processed
Tweets 200000 of 400000 has been processed
Tweets 210000 of 400000 has been processed
Tweets 220000 of 400000 has been processed
Tweets 230000 of 400000 has

In [26]:
%%time
print("Cleaning and parsing the tweets...\n")
for i in range(nums[1],nums[2]):
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[2] ))                                                                    
    clean_tweet_texts.append(tweet_cleaner(df['text'][i]))

Cleaning and parsing the tweets...

Tweets 410000 of 800000 has been processed
Tweets 420000 of 800000 has been processed
Tweets 430000 of 800000 has been processed
Tweets 440000 of 800000 has been processed
Tweets 450000 of 800000 has been processed
Tweets 460000 of 800000 has been processed
Tweets 470000 of 800000 has been processed
Tweets 480000 of 800000 has been processed
Tweets 490000 of 800000 has been processed
Tweets 500000 of 800000 has been processed
Tweets 510000 of 800000 has been processed
Tweets 520000 of 800000 has been processed
Tweets 530000 of 800000 has been processed
Tweets 540000 of 800000 has been processed
Tweets 550000 of 800000 has been processed
Tweets 560000 of 800000 has been processed
Tweets 570000 of 800000 has been processed
Tweets 580000 of 800000 has been processed
Tweets 590000 of 800000 has been processed
Tweets 600000 of 800000 has been processed
Tweets 610000 of 800000 has been processed
Tweets 620000 of 800000 has been processed
Tweets 630000 of 8

In [28]:
%%time
print("Cleaning and parsing the tweets...\n")
for i in range(nums[2],nums[3]):
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[3] ))                                                                 
    clean_tweet_texts.append(tweet_cleaner(df['text'][i]))

Cleaning and parsing the tweets...

Tweets 810000 of 1200000 has been processed
Tweets 820000 of 1200000 has been processed
Tweets 830000 of 1200000 has been processed
Tweets 840000 of 1200000 has been processed
Tweets 850000 of 1200000 has been processed
Tweets 860000 of 1200000 has been processed
Tweets 870000 of 1200000 has been processed
Tweets 880000 of 1200000 has been processed
Tweets 890000 of 1200000 has been processed
Tweets 900000 of 1200000 has been processed
Tweets 910000 of 1200000 has been processed
Tweets 920000 of 1200000 has been processed
Tweets 930000 of 1200000 has been processed
Tweets 940000 of 1200000 has been processed
Tweets 950000 of 1200000 has been processed
Tweets 960000 of 1200000 has been processed
Tweets 970000 of 1200000 has been processed
Tweets 980000 of 1200000 has been processed
Tweets 990000 of 1200000 has been processed
Tweets 1000000 of 1200000 has been processed
Tweets 1010000 of 1200000 has been processed
Tweets 1020000 of 1200000 has been pro

In [29]:
%%time
print("Cleaning and parsing the tweets...\n")
for i in range(nums[3],nums[4]):
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[3] ))                                                                 
    clean_tweet_texts.append(tweet_cleaner(df['text'][i]))

Cleaning and parsing the tweets...

Tweets 1210000 of 1200000 has been processed
Tweets 1220000 of 1200000 has been processed
Tweets 1230000 of 1200000 has been processed
Tweets 1240000 of 1200000 has been processed
Tweets 1250000 of 1200000 has been processed
Tweets 1260000 of 1200000 has been processed
Tweets 1270000 of 1200000 has been processed
Tweets 1280000 of 1200000 has been processed
Tweets 1290000 of 1200000 has been processed
Tweets 1300000 of 1200000 has been processed
Tweets 1310000 of 1200000 has been processed
Tweets 1320000 of 1200000 has been processed
Tweets 1330000 of 1200000 has been processed
Tweets 1340000 of 1200000 has been processed
Tweets 1350000 of 1200000 has been processed
Tweets 1360000 of 1200000 has been processed
Tweets 1370000 of 1200000 has been processed
Tweets 1380000 of 1200000 has been processed
Tweets 1390000 of 1200000 has been processed
Tweets 1400000 of 1200000 has been processed
Tweets 1410000 of 1200000 has been processed
Tweets 1420000 of 1

In [30]:
len(clean_tweet_texts)

1600000

In [31]:
type(clean_tweet_texts)

list

In [33]:
new_clean_tweets_list = []
for i in range(nums[0],nums[1]):
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[1] ))                                                                 
    new_clean_tweets_list.append(clean_tweet_texts[i])
for i in range(nums[3],nums[4]):
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[4] ))                                                                 
    new_clean_tweets_list.append(clean_tweet_texts[i])

Tweets 10000 of 400000 has been processed
Tweets 20000 of 400000 has been processed
Tweets 30000 of 400000 has been processed
Tweets 40000 of 400000 has been processed
Tweets 50000 of 400000 has been processed
Tweets 60000 of 400000 has been processed
Tweets 70000 of 400000 has been processed
Tweets 80000 of 400000 has been processed
Tweets 90000 of 400000 has been processed
Tweets 100000 of 400000 has been processed
Tweets 110000 of 400000 has been processed
Tweets 120000 of 400000 has been processed
Tweets 130000 of 400000 has been processed
Tweets 140000 of 400000 has been processed
Tweets 150000 of 400000 has been processed
Tweets 160000 of 400000 has been processed
Tweets 170000 of 400000 has been processed
Tweets 180000 of 400000 has been processed
Tweets 190000 of 400000 has been processed
Tweets 200000 of 400000 has been processed
Tweets 210000 of 400000 has been processed
Tweets 220000 of 400000 has been processed
Tweets 230000 of 400000 has been processed
Tweets 240000 of 400

In [35]:
new_clean_tweets_list[38000]

'lame oh well i wouldnt be surprised if you could find it online each one is between min long'

In [36]:
df.head()

,sentiment,text,pre_clean_len
0,Negative,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",115
1,Negative,is upset that he can't update his Facebook by ...,111
2,Negative,@Kenichan I dived many times for the ball. Man...,89
3,Negative,my whole body feels itchy and like its on fire,47
4,Negative,"@nationwideclass no, it's not behaving at all....",111


In [37]:
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['sentiment'] = df.sentiment
clean_df.head()

,text,sentiment
0,awww that s a bummer you shoulda got david car...,Negative
1,is upset that he can t update his facebook by ...,Negative
2,i dived many times for the ball managed to sav...,Negative
3,my whole body feels itchy and like its on fire,Negative
4,no it s not behaving at all i m mad why am i h...,Negative


In [54]:
clean_df.to_csv('clean_tweet.csv',encoding='utf-8')

In [14]:
import csv
with open('clean_tweet.csv', 'r') as inp, open('new_clean_tweet_model3.csv', 'w', encoding='utf-8') as out:
    writer = csv.writer(out)
    count = 0
    for row in csv.reader(inp):
        if count == 0:
            print(count)
            writer.writerow(row)
        if count in range(80000,140000):
            print(count)
            writer.writerow(row)
        count = count + 1

0
80000
80001
80002
80003
80004
80005
80006
80007
80008
80009
80010
80011
80012
80013
80014
80015
80016
80017
80018
80019
80020
80021
80022
80023
80024
80025
80026
80027
80028
80029
80030
80031
80032
80033
80034
80035
80036
80037
80038
80039
80040
80041
80042
80043
80044
80045
80046
80047
80048
80049
80050
80051
80052
80053
80054
80055
80056
80057
80058
80059
80060
80061
80062
80063
80064
80065
80066
80067
80068
80069
80070
80071
80072
80073
80074
80075
80076
80077
80078
80079
80080
80081
80082
80083
80084
80085
80086
80087
80088
80089
80090
80091
80092
80093
80094
80095
80096
80097
80098
80099
80100
80101
80102
80103
80104
80105
80106
80107
80108
80109
80110
80111
80112
80113
80114
80115
80116
80117
80118
80119
80120
80121
80122
80123
80124
80125
80126
80127
80128
80129
80130
80131
80132
80133
80134
80135
80136
80137
80138
80139
80140
80141
80142
80143
80144
80145
80146
80147
80148
80149
80150
80151
80152
80153
80154
80155
80156
80157
80158
80159
80160
80161
80162
80163
80164
80165
80

81396
81397
81398
81399
81400
81401
81402
81403
81404
81405
81406
81407
81408
81409
81410
81411
81412
81413
81414
81415
81416
81417
81418
81419
81420
81421
81422
81423
81424
81425
81426
81427
81428
81429
81430
81431
81432
81433
81434
81435
81436
81437
81438
81439
81440
81441
81442
81443
81444
81445
81446
81447
81448
81449
81450
81451
81452
81453
81454
81455
81456
81457
81458
81459
81460
81461
81462
81463
81464
81465
81466
81467
81468
81469
81470
81471
81472
81473
81474
81475
81476
81477
81478
81479
81480
81481
81482
81483
81484
81485
81486
81487
81488
81489
81490
81491
81492
81493
81494
81495
81496
81497
81498
81499
81500
81501
81502
81503
81504
81505
81506
81507
81508
81509
81510
81511
81512
81513
81514
81515
81516
81517
81518
81519
81520
81521
81522
81523
81524
81525
81526
81527
81528
81529
81530
81531
81532
81533
81534
81535
81536
81537
81538
81539
81540
81541
81542
81543
81544
81545
81546
81547
81548
81549
81550
81551
81552
81553
81554
81555
81556
81557
81558
81559
81560
81561
8156

83395
83396
83397
83398
83399
83400
83401
83402
83403
83404
83405
83406
83407
83408
83409
83410
83411
83412
83413
83414
83415
83416
83417
83418
83419
83420
83421
83422
83423
83424
83425
83426
83427
83428
83429
83430
83431
83432
83433
83434
83435
83436
83437
83438
83439
83440
83441
83442
83443
83444
83445
83446
83447
83448
83449
83450
83451
83452
83453
83454
83455
83456
83457
83458
83459
83460
83461
83462
83463
83464
83465
83466
83467
83468
83469
83470
83471
83472
83473
83474
83475
83476
83477
83478
83479
83480
83481
83482
83483
83484
83485
83486
83487
83488
83489
83490
83491
83492
83493
83494
83495
83496
83497
83498
83499
83500
83501
83502
83503
83504
83505
83506
83507
83508
83509
83510
83511
83512
83513
83514
83515
83516
83517
83518
83519
83520
83521
83522
83523
83524
83525
83526
83527
83528
83529
83530
83531
83532
83533
83534
83535
83536
83537
83538
83539
83540
83541
83542
83543
83544
83545
83546
83547
83548
83549
83550
83551
83552
83553
83554
83555
83556
83557
83558
83559
83560
8356

85394
85395
85396
85397
85398
85399
85400
85401
85402
85403
85404
85405
85406
85407
85408
85409
85410
85411
85412
85413
85414
85415
85416
85417
85418
85419
85420
85421
85422
85423
85424
85425
85426
85427
85428
85429
85430
85431
85432
85433
85434
85435
85436
85437
85438
85439
85440
85441
85442
85443
85444
85445
85446
85447
85448
85449
85450
85451
85452
85453
85454
85455
85456
85457
85458
85459
85460
85461
85462
85463
85464
85465
85466
85467
85468
85469
85470
85471
85472
85473
85474
85475
85476
85477
85478
85479
85480
85481
85482
85483
85484
85485
85486
85487
85488
85489
85490
85491
85492
85493
85494
85495
85496
85497
85498
85499
85500
85501
85502
85503
85504
85505
85506
85507
85508
85509
85510
85511
85512
85513
85514
85515
85516
85517
85518
85519
85520
85521
85522
85523
85524
85525
85526
85527
85528
85529
85530
85531
85532
85533
85534
85535
85536
85537
85538
85539
85540
85541
85542
85543
85544
85545
85546
85547
85548
85549
85550
85551
85552
85553
85554
85555
85556
85557
85558
85559
8556

87393
87394
87395
87396
87397
87398
87399
87400
87401
87402
87403
87404
87405
87406
87407
87408
87409
87410
87411
87412
87413
87414
87415
87416
87417
87418
87419
87420
87421
87422
87423
87424
87425
87426
87427
87428
87429
87430
87431
87432
87433
87434
87435
87436
87437
87438
87439
87440
87441
87442
87443
87444
87445
87446
87447
87448
87449
87450
87451
87452
87453
87454
87455
87456
87457
87458
87459
87460
87461
87462
87463
87464
87465
87466
87467
87468
87469
87470
87471
87472
87473
87474
87475
87476
87477
87478
87479
87480
87481
87482
87483
87484
87485
87486
87487
87488
87489
87490
87491
87492
87493
87494
87495
87496
87497
87498
87499
87500
87501
87502
87503
87504
87505
87506
87507
87508
87509
87510
87511
87512
87513
87514
87515
87516
87517
87518
87519
87520
87521
87522
87523
87524
87525
87526
87527
87528
87529
87530
87531
87532
87533
87534
87535
87536
87537
87538
87539
87540
87541
87542
87543
87544
87545
87546
87547
87548
87549
87550
87551
87552
87553
87554
87555
87556
87557
87558
8755

...

129872
129873
129874
129875
129876
129877
129878
129879
129880
129881
129882
129883
129884
129885
129886
129887
129888
129889
129890
129891
129892
129893
129894
129895
129896
129897
129898
129899
129900
129901
129902
129903
129904
129905
129906
129907
129908
129909
129910
129911
129912
129913
129914
129915
129916
129917
129918
129919
129920
129921
129922
129923
129924
129925
129926
129927
129928
129929
129930
129931
129932
129933
129934
129935
129936
129937
129938
129939
129940
129941
129942
129943
129944
129945
129946
129947
129948
129949
129950
129951
129952
129953
129954
129955
129956
129957
129958
129959
129960
129961
129962
129963
129964
129965
129966
129967
129968
129969
129970
129971
129972
129973
129974
129975
129976
129977
129978
129979
129980
129981
129982
129983
129984
129985
129986
129987
129988
129989
129990
129991
129992
129993
129994
129995
129996
129997
129998
129999
130000
130001
130002
130003
130004
130005
130006
130007
130008
130009
130010
130011
130012
130013
130014

131871
131872
131873
131874
131875
131876
131877
131878
131879
131880
131881
131882
131883
131884
131885
131886
131887
131888
131889
131890
131891
131892
131893
131894
131895
131896
131897
131898
131899
131900
131901
131902
131903
131904
131905
131906
131907
131908
131909
131910
131911
131912
131913
131914
131915
131916
131917
131918
131919
131920
131921
131922
131923
131924
131925
131926
131927
131928
131929
131930
131931
131932
131933
131934
131935
131936
131937
131938
131939
131940
131941
131942
131943
131944
131945
131946
131947
131948
131949
131950
131951
131952
131953
131954
131955
131956
131957
131958
131959
131960
131961
131962
131963
131964
131965
131966
131967
131968
131969
131970
131971
131972
131973
131974
131975
131976
131977
131978
131979
131980
131981
131982
131983
131984
131985
131986
131987
131988
131989
131990
131991
131992
131993
131994
131995
131996
131997
131998
131999
132000
132001
132002
132003
132004
132005
132006
132007
132008
132009
132010
132011
132012
132013

133870
133871
133872
133873
133874
133875
133876
133877
133878
133879
133880
133881
133882
133883
133884
133885
133886
133887
133888
133889
133890
133891
133892
133893
133894
133895
133896
133897
133898
133899
133900
133901
133902
133903
133904
133905
133906
133907
133908
133909
133910
133911
133912
133913
133914
133915
133916
133917
133918
133919
133920
133921
133922
133923
133924
133925
133926
133927
133928
133929
133930
133931
133932
133933
133934
133935
133936
133937
133938
133939
133940
133941
133942
133943
133944
133945
133946
133947
133948
133949
133950
133951
133952
133953
133954
133955
133956
133957
133958
133959
133960
133961
133962
133963
133964
133965
133966
133967
133968
133969
133970
133971
133972
133973
133974
133975
133976
133977
133978
133979
133980
133981
133982
133983
133984
133985
133986
133987
133988
133989
133990
133991
133992
133993
133994
133995
133996
133997
133998
133999
134000
134001
134002
134003
134004
134005
134006
134007
134008
134009
134010
134011
134012

135869
135870
135871
135872
135873
135874
135875
135876
135877
135878
135879
135880
135881
135882
135883
135884
135885
135886
135887
135888
135889
135890
135891
135892
135893
135894
135895
135896
135897
135898
135899
135900
135901
135902
135903
135904
135905
135906
135907
135908
135909
135910
135911
135912
135913
135914
135915
135916
135917
135918
135919
135920
135921
135922
135923
135924
135925
135926
135927
135928
135929
135930
135931
135932
135933
135934
135935
135936
135937
135938
135939
135940
135941
135942
135943
135944
135945
135946
135947
135948
135949
135950
135951
135952
135953
135954
135955
135956
135957
135958
135959
135960
135961
135962
135963
135964
135965
135966
135967
135968
135969
135970
135971
135972
135973
135974
135975
135976
135977
135978
135979
135980
135981
135982
135983
135984
135985
135986
135987
135988
135989
135990
135991
135992
135993
135994
135995
135996
135997
135998
135999
136000
136001
136002
136003
136004
136005
136006
136007
136008
136009
136010
136011

137868
137869
137870
137871
137872
137873
137874
137875
137876
137877
137878
137879
137880
137881
137882
137883
137884
137885
137886
137887
137888
137889
137890
137891
137892
137893
137894
137895
137896
137897
137898
137899
137900
137901
137902
137903
137904
137905
137906
137907
137908
137909
137910
137911
137912
137913
137914
137915
137916
137917
137918
137919
137920
137921
137922
137923
137924
137925
137926
137927
137928
137929
137930
137931
137932
137933
137934
137935
137936
137937
137938
137939
137940
137941
137942
137943
137944
137945
137946
137947
137948
137949
137950
137951
137952
137953
137954
137955
137956
137957
137958
137959
137960
137961
137962
137963
137964
137965
137966
137967
137968
137969
137970
137971
137972
137973
137974
137975
137976
137977
137978
137979
137980
137981
137982
137983
137984
137985
137986
137987
137988
137989
137990
137991
137992
137993
137994
137995
137996
137997
137998
137999
138000
138001
138002
138003
138004
138005
138006
138007
138008
138009
138010

In [15]:
with open('clean_tweet.csv', 'r') as inp, open('new_clean_tweet_model3.csv', 'a', encoding='utf-8') as out:
    writer = csv.writer(out)
        
    count = 0
    for row in csv.reader(inp):
        if count in range(1000000,1060000):
            print(count)
            writer.writerow(row)
        count = count + 1

1000000
1000001
1000002
1000003
1000004
1000005
1000006
1000007
1000008
1000009
1000010
1000011
1000012
1000013
1000014
1000015
1000016
1000017
1000018
1000019
1000020
1000021
1000022
1000023
1000024
1000025
1000026
1000027
1000028
1000029
1000030
1000031
1000032
1000033
1000034
1000035
1000036
1000037
1000038
1000039
1000040
1000041
1000042
1000043
1000044
1000045
1000046
1000047
1000048
1000049
1000050
1000051
1000052
1000053
1000054
1000055
1000056
1000057
1000058
1000059
1000060
1000061
1000062
1000063
1000064
1000065
1000066
1000067
1000068
1000069
1000070
1000071
1000072
1000073
1000074
1000075
1000076
1000077
1000078
1000079
1000080
1000081
1000082
1000083
1000084
1000085
1000086
1000087
1000088
1000089
1000090
1000091
1000092
1000093
1000094
1000095
1000096
1000097
1000098
1000099
1000100
1000101
1000102
1000103
1000104
1000105
1000106
1000107
1000108
1000109
1000110
1000111
1000112
1000113
1000114
1000115
1000116
1000117
1000118
1000119
1000120
1000121
1000122
1000123
1000124


1001361
1001362
1001363
1001364
1001365
1001366
1001367
1001368
1001369
1001370
1001371
1001372
1001373
1001374
1001375
1001376
1001377
1001378
1001379
1001380
1001381
1001382
1001383
1001384
1001385
1001386
1001387
1001388
1001389
1001390
1001391
1001392
1001393
1001394
1001395
1001396
1001397
1001398
1001399
1001400
1001401
1001402
1001403
1001404
1001405
1001406
1001407
1001408
1001409
1001410
1001411
1001412
1001413
1001414
1001415
1001416
1001417
1001418
1001419
1001420
1001421
1001422
1001423
1001424
1001425
1001426
1001427
1001428
1001429
1001430
1001431
1001432
1001433
1001434
1001435
1001436
1001437
1001438
1001439
1001440
1001441
1001442
1001443
1001444
1001445
1001446
1001447
1001448
1001449
1001450
1001451
1001452
1001453
1001454
1001455
1001456
1001457
1001458
1001459
1001460
1001461
1001462
1001463
1001464
1001465
1001466
1001467
1001468
1001469
1001470
1001471
1001472
1001473
1001474
1001475
1001476
1001477
1001478
1001479
1001480
1001481
1001482
1001483
1001484
1001485


1003360
1003361
1003362
1003363
1003364
1003365
1003366
1003367
1003368
1003369
1003370
1003371
1003372
1003373
1003374
1003375
1003376
1003377
1003378
1003379
1003380
1003381
1003382
1003383
1003384
1003385
1003386
1003387
1003388
1003389
1003390
1003391
1003392
1003393
1003394
1003395
1003396
1003397
1003398
1003399
1003400
1003401
1003402
1003403
1003404
1003405
1003406
1003407
1003408
1003409
1003410
1003411
1003412
1003413
1003414
1003415
1003416
1003417
1003418
1003419
1003420
1003421
1003422
1003423
1003424
1003425
1003426
1003427
1003428
1003429
1003430
1003431
1003432
1003433
1003434
1003435
1003436
1003437
1003438
1003439
1003440
1003441
1003442
1003443
1003444
1003445
1003446
1003447
1003448
1003449
1003450
1003451
1003452
1003453
1003454
1003455
1003456
1003457
1003458
1003459
1003460
1003461
1003462
1003463
1003464
1003465
1003466
1003467
1003468
1003469
1003470
1003471
1003472
1003473
1003474
1003475
1003476
1003477
1003478
1003479
1003480
1003481
1003482
1003483
1003484


1005359
1005360
1005361
1005362
1005363
1005364
1005365
1005366
1005367
1005368
1005369
1005370
1005371
1005372
1005373
1005374
1005375
1005376
1005377
1005378
1005379
1005380
1005381
1005382
1005383
1005384
1005385
1005386
1005387
1005388
1005389
1005390
1005391
1005392
1005393
1005394
1005395
1005396
1005397
1005398
1005399
1005400
1005401
1005402
1005403
1005404
1005405
1005406
1005407
1005408
1005409
1005410
1005411
1005412
1005413
1005414
1005415
1005416
1005417
1005418
1005419
1005420
1005421
1005422
1005423
1005424
1005425
1005426
1005427
1005428
1005429
1005430
1005431
1005432
1005433
1005434
1005435
1005436
1005437
1005438
1005439
1005440
1005441
1005442
1005443
1005444
1005445
1005446
1005447
1005448
1005449
1005450
1005451
1005452
1005453
1005454
1005455
1005456
1005457
1005458
1005459
1005460
1005461
1005462
1005463
1005464
1005465
1005466
1005467
1005468
1005469
1005470
1005471
1005472
1005473
1005474
1005475
1005476
1005477
1005478
1005479
1005480
1005481
1005482
1005483


1007358
1007359
1007360
1007361
1007362
1007363
1007364
1007365
1007366
1007367
1007368
1007369
1007370
1007371
1007372
1007373
1007374
1007375
1007376
1007377
1007378
1007379
1007380
1007381
1007382
1007383
1007384
1007385
1007386
1007387
1007388
1007389
1007390
1007391
1007392
1007393
1007394
1007395
1007396
1007397
1007398
1007399
1007400
1007401
1007402
1007403
1007404
1007405
1007406
1007407
1007408
1007409
1007410
1007411
1007412
1007413
1007414
1007415
1007416
1007417
1007418
1007419
1007420
1007421
1007422
1007423
1007424
1007425
1007426
1007427
1007428
1007429
1007430
1007431
1007432
1007433
1007434
1007435
1007436
1007437
1007438
1007439
1007440
1007441
1007442
1007443
1007444
1007445
1007446
1007447
1007448
1007449
1007450
1007451
1007452
1007453
1007454
1007455
1007456
1007457
1007458
1007459
1007460
1007461
1007462
1007463
1007464
1007465
1007466
1007467
1007468
1007469
1007470
1007471
1007472
1007473
1007474
1007475
1007476
1007477
1007478
1007479
1007480
1007481
1007482


...

1051836
1051837
1051838
1051839
1051840
1051841
1051842
1051843
1051844
1051845
1051846
1051847
1051848
1051849
1051850
1051851
1051852
1051853
1051854
1051855
1051856
1051857
1051858
1051859
1051860
1051861
1051862
1051863
1051864
1051865
1051866
1051867
1051868
1051869
1051870
1051871
1051872
1051873
1051874
1051875
1051876
1051877
1051878
1051879
1051880
1051881
1051882
1051883
1051884
1051885
1051886
1051887
1051888
1051889
1051890
1051891
1051892
1051893
1051894
1051895
1051896
1051897
1051898
1051899
1051900
1051901
1051902
1051903
1051904
1051905
1051906
1051907
1051908
1051909
1051910
1051911
1051912
1051913
1051914
1051915
1051916
1051917
1051918
1051919
1051920
1051921
1051922
1051923
1051924
1051925
1051926
1051927
1051928
1051929
1051930
1051931
1051932
1051933
1051934
1051935
1051936
1051937
1051938
1051939
1051940
1051941
1051942
1051943
1051944
1051945
1051946
1051947
1051948
1051949
1051950
1051951
1051952
1051953
1051954
1051955
1051956
1051957
1051958
1051959
1051960


1053835
1053836
1053837
1053838
1053839
1053840
1053841
1053842
1053843
1053844
1053845
1053846
1053847
1053848
1053849
1053850
1053851
1053852
1053853
1053854
1053855
1053856
1053857
1053858
1053859
1053860
1053861
1053862
1053863
1053864
1053865
1053866
1053867
1053868
1053869
1053870
1053871
1053872
1053873
1053874
1053875
1053876
1053877
1053878
1053879
1053880
1053881
1053882
1053883
1053884
1053885
1053886
1053887
1053888
1053889
1053890
1053891
1053892
1053893
1053894
1053895
1053896
1053897
1053898
1053899
1053900
1053901
1053902
1053903
1053904
1053905
1053906
1053907
1053908
1053909
1053910
1053911
1053912
1053913
1053914
1053915
1053916
1053917
1053918
1053919
1053920
1053921
1053922
1053923
1053924
1053925
1053926
1053927
1053928
1053929
1053930
1053931
1053932
1053933
1053934
1053935
1053936
1053937
1053938
1053939
1053940
1053941
1053942
1053943
1053944
1053945
1053946
1053947
1053948
1053949
1053950
1053951
1053952
1053953
1053954
1053955
1053956
1053957
1053958
1053959


1055834
1055835
1055836
1055837
1055838
1055839
1055840
1055841
1055842
1055843
1055844
1055845
1055846
1055847
1055848
1055849
1055850
1055851
1055852
1055853
1055854
1055855
1055856
1055857
1055858
1055859
1055860
1055861
1055862
1055863
1055864
1055865
1055866
1055867
1055868
1055869
1055870
1055871
1055872
1055873
1055874
1055875
1055876
1055877
1055878
1055879
1055880
1055881
1055882
1055883
1055884
1055885
1055886
1055887
1055888
1055889
1055890
1055891
1055892
1055893
1055894
1055895
1055896
1055897
1055898
1055899
1055900
1055901
1055902
1055903
1055904
1055905
1055906
1055907
1055908
1055909
1055910
1055911
1055912
1055913
1055914
1055915
1055916
1055917
1055918
1055919
1055920
1055921
1055922
1055923
1055924
1055925
1055926
1055927
1055928
1055929
1055930
1055931
1055932
1055933
1055934
1055935
1055936
1055937
1055938
1055939
1055940
1055941
1055942
1055943
1055944
1055945
1055946
1055947
1055948
1055949
1055950
1055951
1055952
1055953
1055954
1055955
1055956
1055957
1055958


1057833
1057834
1057835
1057836
1057837
1057838
1057839
1057840
1057841
1057842
1057843
1057844
1057845
1057846
1057847
1057848
1057849
1057850
1057851
1057852
1057853
1057854
1057855
1057856
1057857
1057858
1057859
1057860
1057861
1057862
1057863
1057864
1057865
1057866
1057867
1057868
1057869
1057870
1057871
1057872
1057873
1057874
1057875
1057876
1057877
1057878
1057879
1057880
1057881
1057882
1057883
1057884
1057885
1057886
1057887
1057888
1057889
1057890
1057891
1057892
1057893
1057894
1057895
1057896
1057897
1057898
1057899
1057900
1057901
1057902
1057903
1057904
1057905
1057906
1057907
1057908
1057909
1057910
1057911
1057912
1057913
1057914
1057915
1057916
1057917
1057918
1057919
1057920
1057921
1057922
1057923
1057924
1057925
1057926
1057927
1057928
1057929
1057930
1057931
1057932
1057933
1057934
1057935
1057936
1057937
1057938
1057939
1057940
1057941
1057942
1057943
1057944
1057945
1057946
1057947
1057948
1057949
1057950
1057951
1057952
1057953
1057954
1057955
1057956
1057957


1059832
1059833
1059834
1059835
1059836
1059837
1059838
1059839
1059840
1059841
1059842
1059843
1059844
1059845
1059846
1059847
1059848
1059849
1059850
1059851
1059852
1059853
1059854
1059855
1059856
1059857
1059858
1059859
1059860
1059861
1059862
1059863
1059864
1059865
1059866
1059867
1059868
1059869
1059870
1059871
1059872
1059873
1059874
1059875
1059876
1059877
1059878
1059879
1059880
1059881
1059882
1059883
1059884
1059885
1059886
1059887
1059888
1059889
1059890
1059891
1059892
1059893
1059894
1059895
1059896
1059897
1059898
1059899
1059900
1059901
1059902
1059903
1059904
1059905
1059906
1059907
1059908
1059909
1059910
1059911
1059912
1059913
1059914
1059915
1059916
1059917
1059918
1059919
1059920
1059921
1059922
1059923
1059924
1059925
1059926
1059927
1059928
1059929
1059930
1059931
1059932
1059933
1059934
1059935
1059936
1059937
1059938
1059939
1059940
1059941
1059942
1059943
1059944
1059945
1059946
1059947
1059948
1059949
1059950
1059951
1059952
1059953
1059954
1059955
1059956


In [81]:
with open('clean_tweet_800000.csv', 'r') as read_file:
    reader = csv.reader(read_file)
    csvlist = list(reader)
    print(csvlist[349999])
    print(csvlist[410009])
    print(csvlist[1])
    print(csvlist[-1])

['349998', 'i wanna go to ur concert but i can t hope to see you someday', 'Negative']
['1210008', 'omg i just walked through my house and we have little cherub ornaments on photo frames and candles and little random tiny things', 'Positive']
['0', 'awww that s a bummer you shoulda got david carr of third day to do it d', 'Negative']
['1599998', 'happy th birthday to my boo of alll time tupac amaru shakur', 'Positive']


In [82]:
with open('clean_tweet.csv', 'r') as inp, open('clean_tweet_80000.csv', 'w', encoding='utf-8') as out:
    writer = csv.writer(out)
    count = 0
    for row in csv.reader(inp):
        if count in range(0,40000):
            print(count)
            writer.writerow(row)
        count = count + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2366
2367
2368
2369
2370
2371
2372
2373
2374
2375
2376
2377
2378
2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
2399
2400
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415
2416
2417
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429
2430
2431
2432
2433
2434
2435
2436
2437
2438
2439
2440
2441
2442
2443
2444
2445
2446
2447
2448
2449
2450
2451
2452
2453
2454
2455
2456
2457
2458
2459
2460
2461
2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2517
2518
2519
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536
2537
2538
2539
2540
2541
2542
2543
2544
2545
2546
2547
2548
2549
2550
2551
2552
2553
2554
2555
2556
2557
2558
2559
2560
2561
2562
2563
2564
2565


4569
4570
4571
4572
4573
4574
4575
4576
4577
4578
4579
4580
4581
4582
4583
4584
4585
4586
4587
4588
4589
4590
4591
4592
4593
4594
4595
4596
4597
4598
4599
4600
4601
4602
4603
4604
4605
4606
4607
4608
4609
4610
4611
4612
4613
4614
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4630
4631
4632
4633
4634
4635
4636
4637
4638
4639
4640
4641
4642
4643
4644
4645
4646
4647
4648
4649
4650
4651
4652
4653
4654
4655
4656
4657
4658
4659
4660
4661
4662
4663
4664
4665
4666
4667
4668
4669
4670
4671
4672
4673
4674
4675
4676
4677
4678
4679
4680
4681
4682
4683
4684
4685
4686
4687
4688
4689
4690
4691
4692
4693
4694
4695
4696
4697
4698
4699
4700
4701
4702
4703
4704
4705
4706
4707
4708
4709
4710
4711
4712
4713
4714
4715
4716
4717
4718
4719
4720
4721
4722
4723
4724
4725
4726
4727
4728
4729
4730
4731
4732
4733
4734
4735
4736
4737
4738
4739
4740
4741
4742
4743
4744
4745
4746
4747
4748
4749
4750
4751
4752
4753
4754
4755
4756
4757
4758
4759
4760
4761
4762
4763
4764
4765
4766
4767
4768


6568
6569
6570
6571
6572
6573
6574
6575
6576
6577
6578
6579
6580
6581
6582
6583
6584
6585
6586
6587
6588
6589
6590
6591
6592
6593
6594
6595
6596
6597
6598
6599
6600
6601
6602
6603
6604
6605
6606
6607
6608
6609
6610
6611
6612
6613
6614
6615
6616
6617
6618
6619
6620
6621
6622
6623
6624
6625
6626
6627
6628
6629
6630
6631
6632
6633
6634
6635
6636
6637
6638
6639
6640
6641
6642
6643
6644
6645
6646
6647
6648
6649
6650
6651
6652
6653
6654
6655
6656
6657
6658
6659
6660
6661
6662
6663
6664
6665
6666
6667
6668
6669
6670
6671
6672
6673
6674
6675
6676
6677
6678
6679
6680
6681
6682
6683
6684
6685
6686
6687
6688
6689
6690
6691
6692
6693
6694
6695
6696
6697
6698
6699
6700
6701
6702
6703
6704
6705
6706
6707
6708
6709
6710
6711
6712
6713
6714
6715
6716
6717
6718
6719
6720
6721
6722
6723
6724
6725
6726
6727
6728
6729
6730
6731
6732
6733
6734
6735
6736
6737
6738
6739
6740
6741
6742
6743
6744
6745
6746
6747
6748
6749
6750
6751
6752
6753
6754
6755
6756
6757
6758
6759
6760
6761
6762
6763
6764
6765
6766
6767


8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622
8623
8624
8625
8626
8627
8628
8629
8630
8631
8632
8633
8634
8635
8636
8637
8638
8639
8640
8641
8642
8643
8644
8645
8646
8647
8648
8649
8650
8651
8652
8653
8654
8655
8656
8657
8658
8659
8660
8661
8662
8663
8664
8665
8666
8667
8668
8669
8670
8671
8672
8673
8674
8675
8676
8677
8678
8679
8680
8681
8682
8683
8684
8685
8686
8687
8688
8689
8690
8691
8692
8693
8694
8695
8696
8697
8698
8699
8700
8701
8702
8703
8704
8705
8706
8707
8708
8709
8710
8711
8712
8713
8714
8715
8716
8717
8718
8719
8720
8721
8722
8723
8724
8725
8726
8727
8728
8729
8730
8731
8732
8733
8734
8735
8736
8737
8738
8739
8740
8741
8742
8743
8744
8745
8746
8747
8748
8749
8750
8751
8752
8753
8754
8755
8756
8757
8758
8759
8760
8761
8762
8763
8764
8765


...

31554
31555
31556
31557
31558
31559
31560
31561
31562
31563
31564
31565
31566
31567
31568
31569
31570
31571
31572
31573
31574
31575
31576
31577
31578
31579
31580
31581
31582
31583
31584
31585
31586
31587
31588
31589
31590
31591
31592
31593
31594
31595
31596
31597
31598
31599
31600
31601
31602
31603
31604
31605
31606
31607
31608
31609
31610
31611
31612
31613
31614
31615
31616
31617
31618
31619
31620
31621
31622
31623
31624
31625
31626
31627
31628
31629
31630
31631
31632
31633
31634
31635
31636
31637
31638
31639
31640
31641
31642
31643
31644
31645
31646
31647
31648
31649
31650
31651
31652
31653
31654
31655
31656
31657
31658
31659
31660
31661
31662
31663
31664
31665
31666
31667
31668
31669
31670
31671
31672
31673
31674
31675
31676
31677
31678
31679
31680
31681
31682
31683
31684
31685
31686
31687
31688
31689
31690
31691
31692
31693
31694
31695
31696
31697
31698
31699
31700
31701
31702
31703
31704
31705
31706
31707
31708
31709
31710
31711
31712
31713
31714
31715
31716
31717
31718
31719
3172

33553
33554
33555
33556
33557
33558
33559
33560
33561
33562
33563
33564
33565
33566
33567
33568
33569
33570
33571
33572
33573
33574
33575
33576
33577
33578
33579
33580
33581
33582
33583
33584
33585
33586
33587
33588
33589
33590
33591
33592
33593
33594
33595
33596
33597
33598
33599
33600
33601
33602
33603
33604
33605
33606
33607
33608
33609
33610
33611
33612
33613
33614
33615
33616
33617
33618
33619
33620
33621
33622
33623
33624
33625
33626
33627
33628
33629
33630
33631
33632
33633
33634
33635
33636
33637
33638
33639
33640
33641
33642
33643
33644
33645
33646
33647
33648
33649
33650
33651
33652
33653
33654
33655
33656
33657
33658
33659
33660
33661
33662
33663
33664
33665
33666
33667
33668
33669
33670
33671
33672
33673
33674
33675
33676
33677
33678
33679
33680
33681
33682
33683
33684
33685
33686
33687
33688
33689
33690
33691
33692
33693
33694
33695
33696
33697
33698
33699
33700
33701
33702
33703
33704
33705
33706
33707
33708
33709
33710
33711
33712
33713
33714
33715
33716
33717
33718
3371

35552
35553
35554
35555
35556
35557
35558
35559
35560
35561
35562
35563
35564
35565
35566
35567
35568
35569
35570
35571
35572
35573
35574
35575
35576
35577
35578
35579
35580
35581
35582
35583
35584
35585
35586
35587
35588
35589
35590
35591
35592
35593
35594
35595
35596
35597
35598
35599
35600
35601
35602
35603
35604
35605
35606
35607
35608
35609
35610
35611
35612
35613
35614
35615
35616
35617
35618
35619
35620
35621
35622
35623
35624
35625
35626
35627
35628
35629
35630
35631
35632
35633
35634
35635
35636
35637
35638
35639
35640
35641
35642
35643
35644
35645
35646
35647
35648
35649
35650
35651
35652
35653
35654
35655
35656
35657
35658
35659
35660
35661
35662
35663
35664
35665
35666
35667
35668
35669
35670
35671
35672
35673
35674
35675
35676
35677
35678
35679
35680
35681
35682
35683
35684
35685
35686
35687
35688
35689
35690
35691
35692
35693
35694
35695
35696
35697
35698
35699
35700
35701
35702
35703
35704
35705
35706
35707
35708
35709
35710
35711
35712
35713
35714
35715
35716
35717
3571

37551
37552
37553
37554
37555
37556
37557
37558
37559
37560
37561
37562
37563
37564
37565
37566
37567
37568
37569
37570
37571
37572
37573
37574
37575
37576
37577
37578
37579
37580
37581
37582
37583
37584
37585
37586
37587
37588
37589
37590
37591
37592
37593
37594
37595
37596
37597
37598
37599
37600
37601
37602
37603
37604
37605
37606
37607
37608
37609
37610
37611
37612
37613
37614
37615
37616
37617
37618
37619
37620
37621
37622
37623
37624
37625
37626
37627
37628
37629
37630
37631
37632
37633
37634
37635
37636
37637
37638
37639
37640
37641
37642
37643
37644
37645
37646
37647
37648
37649
37650
37651
37652
37653
37654
37655
37656
37657
37658
37659
37660
37661
37662
37663
37664
37665
37666
37667
37668
37669
37670
37671
37672
37673
37674
37675
37676
37677
37678
37679
37680
37681
37682
37683
37684
37685
37686
37687
37688
37689
37690
37691
37692
37693
37694
37695
37696
37697
37698
37699
37700
37701
37702
37703
37704
37705
37706
37707
37708
37709
37710
37711
37712
37713
37714
37715
37716
3771

39550
39551
39552
39553
39554
39555
39556
39557
39558
39559
39560
39561
39562
39563
39564
39565
39566
39567
39568
39569
39570
39571
39572
39573
39574
39575
39576
39577
39578
39579
39580
39581
39582
39583
39584
39585
39586
39587
39588
39589
39590
39591
39592
39593
39594
39595
39596
39597
39598
39599
39600
39601
39602
39603
39604
39605
39606
39607
39608
39609
39610
39611
39612
39613
39614
39615
39616
39617
39618
39619
39620
39621
39622
39623
39624
39625
39626
39627
39628
39629
39630
39631
39632
39633
39634
39635
39636
39637
39638
39639
39640
39641
39642
39643
39644
39645
39646
39647
39648
39649
39650
39651
39652
39653
39654
39655
39656
39657
39658
39659
39660
39661
39662
39663
39664
39665
39666
39667
39668
39669
39670
39671
39672
39673
39674
39675
39676
39677
39678
39679
39680
39681
39682
39683
39684
39685
39686
39687
39688
39689
39690
39691
39692
39693
39694
39695
39696
39697
39698
39699
39700
39701
39702
39703
39704
39705
39706
39707
39708
39709
39710
39711
39712
39713
39714
39715
3971

In [84]:
with open('clean_tweet.csv', 'r') as inp, open('clean_tweet_80000.csv', 'a', encoding='utf-8') as out:
    writer = csv.writer(out)
        
    count = 0
    for row in csv.reader(inp):
        if count in range(800000,840000):
            print(count)
            writer.writerow(row)
        count = count + 1

800000
800001
800002
800003
800004
800005
800006
800007
800008
800009
800010
800011
800012
800013
800014
800015
800016
800017
800018
800019
800020
800021
800022
800023
800024
800025
800026
800027
800028
800029
800030
800031
800032
800033
800034
800035
800036
800037
800038
800039
800040
800041
800042
800043
800044
800045
800046
800047
800048
800049
800050
800051
800052
800053
800054
800055
800056
800057
800058
800059
800060
800061
800062
800063
800064
800065
800066
800067
800068
800069
800070
800071
800072
800073
800074
800075
800076
800077
800078
800079
800080
800081
800082
800083
800084
800085
800086
800087
800088
800089
800090
800091
800092
800093
800094
800095
800096
800097
800098
800099
800100
800101
800102
800103
800104
800105
800106
800107
800108
800109
800110
800111
800112
800113
800114
800115
800116
800117
800118
800119
800120
800121
800122
800123
800124
800125
800126
800127
800128
800129
800130
800131
800132
800133
800134
800135
800136
800137
800138
800139
800140
800141
800142

801517
801518
801519
801520
801521
801522
801523
801524
801525
801526
801527
801528
801529
801530
801531
801532
801533
801534
801535
801536
801537
801538
801539
801540
801541
801542
801543
801544
801545
801546
801547
801548
801549
801550
801551
801552
801553
801554
801555
801556
801557
801558
801559
801560
801561
801562
801563
801564
801565
801566
801567
801568
801569
801570
801571
801572
801573
801574
801575
801576
801577
801578
801579
801580
801581
801582
801583
801584
801585
801586
801587
801588
801589
801590
801591
801592
801593
801594
801595
801596
801597
801598
801599
801600
801601
801602
801603
801604
801605
801606
801607
801608
801609
801610
801611
801612
801613
801614
801615
801616
801617
801618
801619
801620
801621
801622
801623
801624
801625
801626
801627
801628
801629
801630
801631
801632
801633
801634
801635
801636
801637
801638
801639
801640
801641
801642
801643
801644
801645
801646
801647
801648
801649
801650
801651
801652
801653
801654
801655
801656
801657
801658
801659

803016
803017
803018
803019
803020
803021
803022
803023
803024
803025
803026
803027
803028
803029
803030
803031
803032
803033
803034
803035
803036
803037
803038
803039
803040
803041
803042
803043
803044
803045
803046
803047
803048
803049
803050
803051
803052
803053
803054
803055
803056
803057
803058
803059
803060
803061
803062
803063
803064
803065
803066
803067
803068
803069
803070
803071
803072
803073
803074
803075
803076
803077
803078
803079
803080
803081
803082
803083
803084
803085
803086
803087
803088
803089
803090
803091
803092
803093
803094
803095
803096
803097
803098
803099
803100
803101
803102
803103
803104
803105
803106
803107
803108
803109
803110
803111
803112
803113
803114
803115
803116
803117
803118
803119
803120
803121
803122
803123
803124
803125
803126
803127
803128
803129
803130
803131
803132
803133
803134
803135
803136
803137
803138
803139
803140
803141
803142
803143
803144
803145
803146
803147
803148
803149
803150
803151
803152
803153
803154
803155
803156
803157
803158

804515
804516
804517
804518
804519
804520
804521
804522
804523
804524
804525
804526
804527
804528
804529
804530
804531
804532
804533
804534
804535
804536
804537
804538
804539
804540
804541
804542
804543
804544
804545
804546
804547
804548
804549
804550
804551
804552
804553
804554
804555
804556
804557
804558
804559
804560
804561
804562
804563
804564
804565
804566
804567
804568
804569
804570
804571
804572
804573
804574
804575
804576
804577
804578
804579
804580
804581
804582
804583
804584
804585
804586
804587
804588
804589
804590
804591
804592
804593
804594
804595
804596
804597
804598
804599
804600
804601
804602
804603
804604
804605
804606
804607
804608
804609
804610
804611
804612
804613
804614
804615
804616
804617
804618
804619
804620
804621
804622
804623
804624
804625
804626
804627
804628
804629
804630
804631
804632
804633
804634
804635
804636
804637
804638
804639
804640
804641
804642
804643
804644
804645
804646
804647
804648
804649
804650
804651
804652
804653
804654
804655
804656
804657

806513
806514
806515
806516
806517
806518
806519
806520
806521
806522
806523
806524
806525
806526
806527
806528
806529
806530
806531
806532
806533
806534
806535
806536
806537
806538
806539
806540
806541
806542
806543
806544
806545
806546
806547
806548
806549
806550
806551
806552
806553
806554
806555
806556
806557
806558
806559
806560
806561
806562
806563
806564
806565
806566
806567
806568
806569
806570
806571
806572
806573
806574
806575
806576
806577
806578
806579
806580
806581
806582
806583
806584
806585
806586
806587
806588
806589
806590
806591
806592
806593
806594
806595
806596
806597
806598
806599
806600
806601
806602
806603
806604
806605
806606
806607
806608
806609
806610
806611
806612
806613
806614
806615
806616
806617
806618
806619
806620
806621
806622
806623
806624
806625
806626
806627
806628
806629
806630
806631
806632
806633
806634
806635
806636
806637
806638
806639
806640
806641
806642
806643
806644
806645
806646
806647
806648
806649
806650
806651
806652
806653
806654
806655

...

831999
832000
832001
832002
832003
832004
832005
832006
832007
832008
832009
832010
832011
832012
832013
832014
832015
832016
832017
832018
832019
832020
832021
832022
832023
832024
832025
832026
832027
832028
832029
832030
832031
832032
832033
832034
832035
832036
832037
832038
832039
832040
832041
832042
832043
832044
832045
832046
832047
832048
832049
832050
832051
832052
832053
832054
832055
832056
832057
832058
832059
832060
832061
832062
832063
832064
832065
832066
832067
832068
832069
832070
832071
832072
832073
832074
832075
832076
832077
832078
832079
832080
832081
832082
832083
832084
832085
832086
832087
832088
832089
832090
832091
832092
832093
832094
832095
832096
832097
832098
832099
832100
832101
832102
832103
832104
832105
832106
832107
832108
832109
832110
832111
832112
832113
832114
832115
832116
832117
832118
832119
832120
832121
832122
832123
832124
832125
832126
832127
832128
832129
832130
832131
832132
832133
832134
832135
832136
832137
832138
832139
832140
832141

833998
833999
834000
834001
834002
834003
834004
834005
834006
834007
834008
834009
834010
834011
834012
834013
834014
834015
834016
834017
834018
834019
834020
834021
834022
834023
834024
834025
834026
834027
834028
834029
834030
834031
834032
834033
834034
834035
834036
834037
834038
834039
834040
834041
834042
834043
834044
834045
834046
834047
834048
834049
834050
834051
834052
834053
834054
834055
834056
834057
834058
834059
834060
834061
834062
834063
834064
834065
834066
834067
834068
834069
834070
834071
834072
834073
834074
834075
834076
834077
834078
834079
834080
834081
834082
834083
834084
834085
834086
834087
834088
834089
834090
834091
834092
834093
834094
834095
834096
834097
834098
834099
834100
834101
834102
834103
834104
834105
834106
834107
834108
834109
834110
834111
834112
834113
834114
834115
834116
834117
834118
834119
834120
834121
834122
834123
834124
834125
834126
834127
834128
834129
834130
834131
834132
834133
834134
834135
834136
834137
834138
834139
834140

835997
835998
835999
836000
836001
836002
836003
836004
836005
836006
836007
836008
836009
836010
836011
836012
836013
836014
836015
836016
836017
836018
836019
836020
836021
836022
836023
836024
836025
836026
836027
836028
836029
836030
836031
836032
836033
836034
836035
836036
836037
836038
836039
836040
836041
836042
836043
836044
836045
836046
836047
836048
836049
836050
836051
836052
836053
836054
836055
836056
836057
836058
836059
836060
836061
836062
836063
836064
836065
836066
836067
836068
836069
836070
836071
836072
836073
836074
836075
836076
836077
836078
836079
836080
836081
836082
836083
836084
836085
836086
836087
836088
836089
836090
836091
836092
836093
836094
836095
836096
836097
836098
836099
836100
836101
836102
836103
836104
836105
836106
836107
836108
836109
836110
836111
836112
836113
836114
836115
836116
836117
836118
836119
836120
836121
836122
836123
836124
836125
836126
836127
836128
836129
836130
836131
836132
836133
836134
836135
836136
836137
836138
836139

837966
837967
837968
837969
837970
837971
837972
837973
837974
837975
837976
837977
837978
837979
837980
837981
837982
837983
837984
837985
837986
837987
837988
837989
837990
837991
837992
837993
837994
837995
837996
837997
837998
837999
838000
838001
838002
838003
838004
838005
838006
838007
838008
838009
838010
838011
838012
838013
838014
838015
838016
838017
838018
838019
838020
838021
838022
838023
838024
838025
838026
838027
838028
838029
838030
838031
838032
838033
838034
838035
838036
838037
838038
838039
838040
838041
838042
838043
838044
838045
838046
838047
838048
838049
838050
838051
838052
838053
838054
838055
838056
838057
838058
838059
838060
838061
838062
838063
838064
838065
838066
838067
838068
838069
838070
838071
838072
838073
838074
838075
838076
838077
838078
838079
838080
838081
838082
838083
838084
838085
838086
838087
838088
838089
838090
838091
838092
838093
838094
838095
838096
838097
838098
838099
838100
838101
838102
838103
838104
838105
838106
838107
838108

839495
839496
839497
839498
839499
839500
839501
839502
839503
839504
839505
839506
839507
839508
839509
839510
839511
839512
839513
839514
839515
839516
839517
839518
839519
839520
839521
839522
839523
839524
839525
839526
839527
839528
839529
839530
839531
839532
839533
839534
839535
839536
839537
839538
839539
839540
839541
839542
839543
839544
839545
839546
839547
839548
839549
839550
839551
839552
839553
839554
839555
839556
839557
839558
839559
839560
839561
839562
839563
839564
839565
839566
839567
839568
839569
839570
839571
839572
839573
839574
839575
839576
839577
839578
839579
839580
839581
839582
839583
839584
839585
839586
839587
839588
839589
839590
839591
839592
839593
839594
839595
839596
839597
839598
839599
839600
839601
839602
839603
839604
839605
839606
839607
839608
839609
839610
839611
839612
839613
839614
839615
839616
839617
839618
839619
839620
839621
839622
839623
839624
839625
839626
839627
839628
839629
839630
839631
839632
839633
839634
839635
839636
839637

In [87]:
string1 = '0.06 is my key'
new_string = string1.apply(lambda x: x.lower())
print(new_string)

AttributeError: 'str' object has no attribute 'apply'